In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

Note - tried scrapping 1000 restaurants. For the review links function, after 25 mins the extraction stopped as the connection was forcibly closed by the remote host. Was extracting too much information :)

In [2]:
## DOMAIN = 'https://tabelog.com/'
targeted_region = 'hokkaido'  # tokyo, osaka, kyoto, hokkaido
RESTAURANT_URL = 'https://tabelog.com/en/' + targeted_region + '/rstLst/'
page = 5 # number of restaurants, 1 page = 20 restaurants

print("The restaurant URL is ", RESTAURANT_URL)
print("The number of restaurants to scrap", page*20)

The restaurant URL is  https://tabelog.com/en/hokkaido/rstLst/
The number of restaurants to scrap 100


# Get Restaurant Details Data

In [3]:
"""
This function helps to get the restaurant URLs
"""
def get_url(page):
    url_list = []
    
    for i in range(page):
        url = RESTAURANT_URL + str(i) + '/'
        #print("URL is ",url)
        
        # Send a request to the URL
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        #print("Type of soup is",type(soup))
        #print("Soup is",soup)
       
        
        # grab "https://tabelog.com/en/tokyo/A1317/A131712/13171774/"
        restaurants = soup.findAll("a", {"list-rst__rst-name-target cpy-rst-name"},href=True) 
        #print("restaurants are:", restaurants)

        for res in restaurants:
            url_list.append(res['href'])
    return url_list

In [4]:
%%time
restaurant_links = get_url(page)

CPU times: total: 469 ms
Wall time: 11.4 s


In [5]:
restaurant_links 

['https://tabelog.com/en/hokkaido/A0101/A010103/1072073/',
 'https://tabelog.com/en/hokkaido/A0101/A010103/1064993/',
 'https://tabelog.com/en/hokkaido/A0101/A010103/1003634/',
 'https://tabelog.com/en/hokkaido/A0101/A010103/1004125/',
 'https://tabelog.com/en/hokkaido/A0101/A010103/1028665/',
 'https://tabelog.com/en/hokkaido/A0101/A010103/1069091/',
 'https://tabelog.com/en/hokkaido/A0101/A010103/1008057/',
 'https://tabelog.com/en/hokkaido/A0101/A010103/1073252/',
 'https://tabelog.com/en/hokkaido/A0101/A010103/1077231/',
 'https://tabelog.com/en/hokkaido/A0101/A010104/1000249/',
 'https://tabelog.com/en/hokkaido/A0101/A010103/1071005/',
 'https://tabelog.com/en/hokkaido/A0104/A010401/1068350/',
 'https://tabelog.com/en/hokkaido/A0101/A010103/1044921/',
 'https://tabelog.com/en/hokkaido/A0104/A010403/1058891/',
 'https://tabelog.com/en/hokkaido/A0105/A010501/1028734/',
 'https://tabelog.com/en/hokkaido/A0105/A010501/1060964/',
 'https://tabelog.com/en/hokkaido/A0101/A010103/1057421/

In [64]:
print(len(restaurant_links))
restaurant_links[0]

100


'https://tabelog.com/en/hokkaido/A0101/A010103/1028665/'

In [47]:
""" 
This function will store the restaurant details. The details include name, categories, phone number, reservation, transport, address, opening hours, average price, average price based on reviews, payments, service charge, receipt
"""

def scrape(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # <table class="c-table rd-detail-info">
    table = soup.find("table", class_ = "c-table c-table--form rstinfo-table__table") 
    #print(table)
    rows = table.tbody.find_all('tr')
    
    res_info = {}
    for row in rows:
        res_info[row.find('th').text.strip()] = row.find('td').text.strip().replace('\n','')
#     print(res_info)
    
    return res_info

In [48]:
%%time
restaurants_info = []
for url in restaurant_links:
    cur_restaurant = scrape(url)
    restaurants_info.append(cur_restaurant)

CPU times: total: 1.45 s
Wall time: 3min 5s


In [49]:
df = pd.DataFrame.from_dict(restaurants_info)
print(df.shape)
df.head(1)

(100, 13)


,Restaurant name,Categories,Phone number (for reservation and inquiry),Reservation availability,Address,Transportation,Opening hours,Average price,Average price（Based on reviews）,Payment methods,Receipt (Qualified simple invoice),Service charge & fee,Awards & Recognitions
0,Kyoto Sushi Momonoki,"Sushi, Seafood",050-5600-6934,Reservations available ...,京都府京都市下京区真苧屋町99 ブーケガルニ B1F ...,Subway Toshima Line / Kyoto Station 4 minutes ...,"Mon, Tue, Wed, Thu, Fri ...","JPY 10,000 - JPY 14,999JPY 4,000 - JPY 4,999","JPY 20,000 - JPY 29,999JPY 6,000 - JPY 7,999Vi...",Credit card accepted ...,Qualified invoice (receipt) availableRegistrat...,Service charge 10% ...,NaN


In [50]:
#df.to_csv("output/"+targeted_region+"_restaurant_data.csv", index=False)

# Get the Restaurant Menu Data

In [65]:
restaurant_menu_links = [link.rsplit('/', 1)[0] + '/dtlmenu/' for link in restaurant_links]

In [66]:
restaurant_menu_links[42]

'https://tabelog.com/en/hokkaido/A0101/A010101/1020704/dtlmenu/'

In [67]:
restaurant_menu_links[12]

'https://tabelog.com/en/hokkaido/A0101/A010102/1078638/dtlmenu/'

In [68]:
len(restaurant_menu_links)

100

In [69]:

# Function to scrape restaurant name and menu titles from a single URL
def get_menu_data(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find restaurant name using the specified class
        restaurant_name_tag = soup.find('h2', class_='display-name')
        restaurant_name = restaurant_name_tag.get_text(strip=True) if restaurant_name_tag else "No name found"
        
        # Find all menu titles with the specified class
        titles = soup.find_all('p', class_='rstdtl-menu-lst__menu-title')
        #print("titles", titles)
        menu_titles = [title.get_text(strip=True) for title in titles]
        
        return restaurant_name, menu_titles
    else:
        #print("inside else")
        return "No response", []

# # Iterate through all links and print the restaurant name and titles
# for link in restaurant_menu_links:
#     print(f"Scraping data for {link}:")
#     restaurant_name, titles = get_menu_data(link)
#     print(f"Restaurant: {restaurant_name}")
#     print("Menu titles:")
#     for title in titles:
#         print(f"- {title}")
#     print()


In [70]:
%%time

# Create an empty list to store the data
menu_data = []

# Iterate through all links and store the restaurant name and menu titles in the list
for link in restaurant_menu_links:
    restaurant_name, titles = get_menu_data(link)
    
    # If no titles are found, append a row with just the restaurant name and blank dishes
    if not titles:
        menu_data.append({
            'restaurant_name': restaurant_name,
            'dishes': ''
        })
    else:
        # Store each menu title along with the restaurant name
        for title in titles:
            menu_data.append({
                'restaurant_name': restaurant_name,
                'dishes': title
            })

# Convert the list to a DataFrame
menu_df = pd.DataFrame(menu_data)


CPU times: total: 2.23 s
Wall time: 2min 47s


In [76]:
#menu_df["location"] = targeted_region

In [74]:
print(menu_df.shape)
menu_df.head()

(4463, 3)


,restaurant_name,dishes,location
0,AKI NAGAO,akinagao Chef'sCourse Akinagao selection Course,hokkaido
1,L'Oiseau par Matsunaga,,hokkaido
2,HAJIME,Cow tongue tongue shabu-shabu shabu shabu,hokkaido
3,HAJIME,Lamb shabu set Lamb shabu set,hokkaido
4,HAJIME,Cow tongue tongue shabu-shabu,hokkaido


In [75]:
menu_df.to_csv("output/"+targeted_region+"_restaurant_menu_data.csv", index=False)

# Get the Restaurant Reviews Data 

In [9]:
""" 
This function will give us links to reviews page of the restaurants
"""


def get_url2(my_url):
    r = requests.get(my_url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # Find the 'a' tag with the class 'c-link-circle'
    link = soup.find('a', class_='c-link-circle')

    # Extract the 'href' attribute only
    if link:
        review_url = link.get('href')

        # everything after the last forward slash should be replaced with 'dtlrvwlist'
        if 'party' in review_url or review_url.endswith('/'):
            review_url = review_url.rsplit('/party/', 1)[0] + '/dtlrvwlst/'
        else:
            review_url = None # or return the review_url itself

        #print("Modified Review link:", review_url)
        return review_url
    else:
        #print("No review link found.")
        return None


In [10]:
%%time
review_links = []
for my_url in restaurant_links:
    cur_review = get_url2(my_url)
    review_links.append(cur_review)

CPU times: total: 1.52 s
Wall time: 2min 49s


In [11]:
print(len(review_links))
review_links[0]

100


'https://tabelog.com/en/tokyo/A1306/A130602/13042979/dtlrvwlst/'

In [12]:
"""" 
Since, full review is not in the links grabbed above, therefore we will get the full review by clicking on the view_more button. This information will be in a new URL.
"""

def get_url3(my_url):
    # Send a request to the URL
    r = requests.get(my_url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # Find all 'span' elements with the class 'rvw-showall-trigger__target js-link-bookmark-detail'
    review_links = soup.find_all('span', class_='rvw-showall-trigger__target js-link-bookmark-detail')

    # Extract the 'data-detail-url' attribute for each review
    review_urls = []
    for review in review_links:
        data_url = review.get('data-detail-url')
        if data_url:
            review_urls.append(data_url)

    return(review_urls)


In [13]:
%%time

review_link_info_page = []

for my_url in review_links:
    cur_review_url = get_url3(my_url)
    review_link_info_page .append(cur_review_url)

CPU times: total: 5.08 s
Wall time: 3min 30s


In [14]:
print(len(review_link_info_page ))
print(len(review_link_info_page[60]))

100
24


In [15]:
review_link_info_page[60]

['https://tabelog.com/en/tokyo/A1303/A130301/13289319/dtlrvwlst/B472913068/?smp=0&use_type=0',
 'https://tabelog.com/en/tokyo/A1303/A130301/13289319/dtlrvwlst/B482102684/?smp=0&use_type=0',
 'https://tabelog.com/en/tokyo/A1303/A130301/13289319/dtlrvwlst/B472972295/?smp=0&use_type=0',
 'https://tabelog.com/en/tokyo/A1303/A130301/13289319/dtlrvwlst/B477251744/?smp=0&use_type=0',
 'https://tabelog.com/en/tokyo/A1303/A130301/13289319/dtlrvwlst/B474609437/?smp=0&use_type=0',
 'https://tabelog.com/en/tokyo/A1303/A130301/13289319/dtlrvwlst/B492575294/?smp=0&use_type=0',
 'https://tabelog.com/en/tokyo/A1303/A130301/13289319/dtlrvwlst/B492645672/?smp=0&use_type=0',
 'https://tabelog.com/en/tokyo/A1303/A130301/13289319/dtlrvwlst/B492673452/?smp=0&use_type=0',
 'https://tabelog.com/en/tokyo/A1303/A130301/13289319/dtlrvwlst/B484356982/?smp=0&use_type=0',
 'https://tabelog.com/en/tokyo/A1303/A130301/13289319/dtlrvwlst/B491688898/?smp=0&use_type=0',
 'https://tabelog.com/en/tokyo/A1303/A130301/13289

In [16]:
""" 
Grab all the information from the new review URL. This includes - the review, the restaurant name and ratings
"""


def get_reviews(my_url):

    r = requests.get(my_url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # Extract the restaurant name from <h2 class="display-name">
    restaurant_name_tag = soup.find('h2', class_='display-name')
    if restaurant_name_tag:
        restaurant_name = restaurant_name_tag.get_text(strip=True)
        #print("Restaurant Name:", restaurant_name)
    else:
        restaurant_name = None
        #print("No restaurant name found.")

    # Extract the total restaurant rating from <ul class="rdheader-counts">
    total_restaurant_rating_tag = soup.find('ul', class_='rdheader-counts')
    if total_restaurant_rating_tag :
        total_restaurant_rating= total_restaurant_rating_tag.get_text(strip=True)
        #print("Total Rating:", total_restaurant_rating)
    else:
        total_restaurant_rating = None
        #print("No total rating found.")

    # Extract the ratings div
    individual_rating_div = soup.find('div', class_='rvw-item__single-ratings')
    if individual_rating_div:
        individual_rating_info = individual_rating_div.get_text(strip=True)
        #print("Individual Rating Info:", individual_rating_info)
    else:
        individual_rating_info = None
        #print("No individual rating info found.")

    # Extract the comment div
    comment_div = soup.find('div', class_='rvw-item__rvw-comment rvw-item__rvw-comment--custom')
    if comment_div:
        comment_info = comment_div.get_text(strip=True)
        #print("Comment Info:", comment_info)
    else:
        comment_info = None
        #print("No comment info found.")

    return {
        'restaurant_name': restaurant_name,
        'restaurant_rating': total_restaurant_rating,
        'individual_rating': individual_rating_info,
        'comments': comment_info
    }


In [19]:
%%time

detailed_reviews = []

for review_data in review_link_info_page:
    for i in review_data:
        review_details = get_reviews(i)
        detailed_reviews.append(review_details)

CPU times: total: 31.8 s
Wall time: 43min 55s


In [20]:
#pd.DataFrame(detailed_reviews).head()
final_review_df = pd.DataFrame.from_dict(detailed_reviews)
print(final_review_df.shape)
final_review_df.head()


(1590, 4)


,restaurant_name,restaurant_rating,individual_rating,comments
0,Hokkaido Shabushabu Pokke Dai3 Gurin Biru Ten,3.2123reviews,3.6Food and taste-Service-Atmosphere-Cost perf...,All-you-can-eat lamb and pork. And all you can...
1,Hokkaido Shabushabu Pokke Dai3 Gurin Biru Ten,3.2123reviews,1.0Food and taste3.0Service1.0Atmosphere1.0Cos...,The restaurant I was planning to go to was clo...
2,Hokkaido Shabushabu Pokke Dai3 Gurin Biru Ten,3.2123reviews,4.0Food and taste4.0Service3.6Atmosphere3.6Cos...,My favorite Pokke-san after a long time. Today...
3,Hokkaido Shabushabu Pokke Dai3 Gurin Biru Ten,3.2123reviews,3.3Food and taste3.3Service3.0Atmosphere3.0Cos...,I came to visit after realizing that I had nev...
4,Hokkaido Shabushabu Pokke Dai3 Gurin Biru Ten,3.2123reviews,3.0Food and taste3.0Service3.0Atmosphere2.0Cos...,Assorted vegetables Meat I think the price is ...


In [21]:
#final_review_df.to_csv("output/"+targeted_region+"_restaurant_review_data.csv", index=False)